I try to model a **conventional car**: The rear axle is driven, the front axle does the steering.\
No speed possible perpendicular to the wheels.\
The car must drive from A to B, using minimal energy, and it must stay on a road.\
It should stop at an interval (gate) $((B|final_u), (B|final_o))$

In [1]:
import sympy.physics.mechanics as me
import time
import numpy as np
import sympy as sm
from scipy.interpolate import CubicSpline

from opty.direct_collocation import Problem
from opty.utils import parse_free, create_objective_function
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.animation import FuncAnimation

Set up **Kane's equations of motion**

In [2]:
start = time.time()

N, A0, Ab, Af = sm.symbols('N A0 Ab Af', cls= me.ReferenceFrame)
t = me.dynamicsymbols._t
O, Pb, Dmc, Pf = sm.symbols('O Pb Dmc Pf', cls= me.Point)
O.set_vel(N, 0)

q0, qf  = me.dynamicsymbols('q_0 q_f')
u0, uf  = me.dynamicsymbols('u_0 u_f')
x, y    = me.dynamicsymbols('x y')
ux, uy  = me.dynamicsymbols('u_x u_y')
Tf, Fb  = me.dynamicsymbols('T_f F_b')

reibung = sm.symbols('reibung')

l, m0, mb, mf, iZZ0, iZZb, iZZf = sm.symbols('l m0 mb mf iZZ0, iZZb, iZZf')

A0.orient_axis(N, q0, N.z)
A0.set_ang_vel(N, u0 * N.z)

Ab.orient_axis(A0, 0, N.z)

Af.orient_axis(A0, qf, N.z)
rot = Af.ang_vel_in(N)
Af.set_ang_vel(N, uf * N.z)
rot1 = Af.ang_vel_in(N)

Pf.set_pos(O, x * N.x + y * N.y)
Pf.set_vel(N, ux * N.x + uy * N.y)

Pb.set_pos(Pf, -l * A0.y)
Pb.v2pt_theory(Pf, N, A0)

Dmc.set_pos(Pf, -l/2 * A0.y)
Dmc.v2pt_theory(Pf, N, A0)

# No speed perpendicular to the wheels
vel1 = me.dot(Pb.vel(N), Ab.x) - 0
vel2 = me.dot(Pf.vel(N), Af.x) - 0

I0 = me.inertia(A0, 0, 0, iZZ0)
body0 = me.RigidBody('body0', Dmc, A0, m0, (I0, Dmc))
Ib = me.inertia(Ab, 0, 0, iZZb)
bodyb = me.RigidBody('bodyb', Pb, Ab, mb, (Ib, Pb))
If = me.inertia(Af, 0, 0, iZZf)
bodyf = me.RigidBody('bodyf', Pf, Af, mf, (If, Pf))
BODY = [body0, bodyb, bodyf]


# Forces
FL = [(Pb, Fb * Ab.y), (Af, Tf * N.z), (Dmc, -reibung * Dmc.vel(N))]

kd = sm.Matrix([ux - x.diff(t), uy - y.diff(t), u0 - q0.diff(t),
                me.dot(rot1- rot, N.z)])
speed_constr = sm.Matrix([vel1, vel2])

q_ind = [x, y, q0, qf]
u_ind = [u0, uf]
u_dep = [ux, uy]

# Create the KanesMethod object
KM = me.KanesMethod(
                    N, q_ind=q_ind, u_ind=u_ind,
                    kd_eqs=kd,
                    u_dependent=u_dep,
                    velocity_constraints=speed_constr,
                    )
(fr, frstar) = KM.kanes_equations(BODY, FL)
eom = fr + frstar
eom = kd.col_join(eom)
eom = eom.col_join(speed_constr)

Set up so the car stays on the street\
The street is described by two functions in the X / Y plane f(x), g(x)
with f(x) < g(x) for all x. For *number* of points $P(x_i. y_i) \in $ car it must hold:\
$ f(x_i) < y_i < g(x_i)$\
As *opty* at present does not allow inequalities in the eoms, I circumvent this by
introducing additional state variables.

In [3]:
XX, a, b, c, d = sm.symbols('XX a b c d')
def street(XX, a, b, c):
    return a*sm.sin(b*XX) + c

number = 4
py_upper = me.dynamicsymbols(f'py_upper:{number}')
py_lower = me.dynamicsymbols(f'py_lower:{number}')

park1y = Pf.pos_from(O).dot(N.y)
park2y = Pb.pos_from(O).dot(N.y)
park1x = Pf.pos_from(O).dot(N.x)
park2x = Pb.pos_from(O).dot(N.x)

delta_x = np.linspace(park1x, park2x, number)
delta_y = np.linspace(park1y, park2y, number)

delta_p_u = [delta_y[i] - street(delta_x[i], a, b, c) for i in range(number)]
delta_p_l = [-delta_y[i] + street(delta_x[i], a, b, c+d) for i in range(number)]

eom_add = sm.Matrix([
    *[-py_upper[i] + delta_p_u[i] for i in range(number)],
    *[-py_lower[i] + delta_p_l[i] for i in range(number)],
])

eom = eom.col_join(eom_add)

*opty* presently does not support bounds for instance constraints.\
I circumvent this as follows:\
I add two state variables $top$ and $bottom$ and two specified symbolds $top_h$
and $bottom_h$\
Say, we need $a \leq y(t_x) \leq b, a, b > 0 $, then I add this to the equations of motion:

$top = top_h \cdot y(t_x)$\
$bottom = bottom_h \cdot y(t_x)$

Then I bound $top_h \in (1.0, \inf)$ and 
$bottom_h \in (0.0, 1.0)$, and put instance constraints $top(t_x) = b, 
bottom(t_x) = a$\
This forces $a \leq y(t_x) \leq b$


In [ ]:
top, bottom = me.dynamicsymbols('top, bottom')
top_h, bottom_h = me.dynamicsymbols('top_h, bottom_h')

eom_add = sm.Matrix([
    -top + park1y * top_h,
    -bottom + park1y * bottom_h,
])

eom = eom.col_join(eom_add)

print(f'eom too large to print out. Its shape is {eom.shape} and it has ' +
      f'{sm.count_ops(eom)} operations')

Set up the machinery for **opty** and **solve** the optimization.\
If needed, one can iterate, using the solution of iteration i as initial_guess for iteration i + 1.\
As the convergence is very slow, I use the result of a previous run as initial guess.

In [ ]:

state_symbols = [x, y, q0, qf, ux, uy, u0, uf] + py_upper + py_lower + [top, bottom]

laenge = len(state_symbols)
constant_symbols = (l, m0, mb, mf, iZZ0, iZZb, iZZf, reibung, a, b, c, d)
specified_symbols = [Fb, Tf] +[top_h, bottom_h]
unknown_symbols = ()

# Specify the known system parameters.
final_o = 4.0
final_u = 2.0
par_map = {}
par_map[m0] = 1.0
par_map[mb] = 0.5
par_map[mf] = 0.5
par_map[iZZ0] = 1.
par_map[iZZb] = 0.5
par_map[iZZf] = 0.5
par_map[l] = 3.0
par_map[reibung] = 0.5
# below for the shape of the street
par_map[a] = 3.5
par_map[b] = 0.5
par_map[c] = 4.0
par_map[d] = 4.5

t0, tf = 0.0, 10.0
num_nodes = 601
interval_value = (tf - t0)/(num_nodes - 1)

objective = sm.Integral(Tf**2 + 1.e2*Fb**2, t)
obj, obj_grad = create_objective_function(
    objective,
    state_symbols,
    specified_symbols,
    tuple(),
    num_nodes,
    interval_value,
)

instance_constraints = (
        x.func(t0) + 10.0,
        ux.func(t0),
        uy.func(t0),
        u0.func(t0),
        uf.func(t0),

        x.func(tf) - 10.0,
        ux.func(tf),
        uy.func(tf),
        top.func(tf) - final_o,
        bottom.func(tf) - final_u,
)

grenze = 100.0
grenze1 = 5.0
delta = np.pi/4.0
bounds1 = {
        Fb: (-grenze, grenze),
        Tf: (-grenze, grenze),
        # restrict the steering angle to avoid locking
        qf: (-np.pi/2. + delta, np.pi/2. - delta),
        x: (-14, 14),
        y: (0.0, 25),
        top_h: (1.0, 10.0),
        bottom_h: (0.0, 1.0),
}

bounds2 = {py_upper[i]: (0, 10.0) for i in range(number)}
bounds3 = {py_lower[i]: (0.0, 10.0) for i in range(number)}
bounds = {**bounds1, **bounds2, **bounds3}

prob = Problem(
        obj,
        obj_grad,
        eom,
        state_symbols,
        num_nodes,
        interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
        bounds=bounds,
)

initial_guess = np.ones(prob.num_free) * 0.01
initial_guess = np.load('car_on_street_gate_solution.npy')
prob.add_option('max_iter', 3000)
for i in range(1):
# Find the optimal solution.
    solution, info = prob.solve(initial_guess)
    initial_guess = solution
    print(f'{i+1} - th iteration')
    print('message from optimizer:', info['status_msg'])
    print('Iterations needed',len(prob.obj_value))
    print(f"objective value {info['obj_val']:.3e} \n")

prob.plot_objective_value()

Plot the constraint violations.

In [ ]:
prob.plot_constraint_violations(solution)

Plot generalized coordinates / speeds and forces / torques

In [ ]:
prob.plot_trajectories(solution)

aminate the **car**\
The green arrows symbolize the forces which opty calculated to drive the car. They are perpendicular to the rear axle.

In [ ]:
fps = 15
street_lam = sm.lambdify((x, a, b, c), street(x, a, b, c))

def add_point_to_data(line, x, y):
# to trace the path of the point. Copied from Timo.
    old_x, old_y = line.get_data()
    line.set_data(np.append(old_x, x), np.append(old_y, y))


state_vals, input_vals, _ = parse_free(solution, len(state_symbols),
                len(specified_symbols), num_nodes)
t_arr = np.linspace(t0, tf, num_nodes)
state_sol = CubicSpline(t_arr, state_vals.T)
input_sol = CubicSpline(t_arr, input_vals.T)

# create additional points for the axles
Pbl, Pbr, Pfl, Pfr = sm.symbols('Pbl Pbr Pfl Pfr', cls= me.Point)

# end points of the force, length of the axles
Fbq = me.Point('Fbq')
la = sm.symbols('la')
fb, tq = sm.symbols('f_b, t_q')

Pbl.set_pos(Pb, -la/2 * Ab.x)
Pbr.set_pos(Pb, la/2 * Ab.x)
Pfl.set_pos(Pf, -la/2 * Af.x)
Pfr.set_pos(Pf, la/2 * Af.x)

Fbq.set_pos(Pb, fb * Ab.y)

coordinates = Pb.pos_from(O).to_matrix(N)
for point in (Dmc, Pf, Pbl, Pbr, Pfl, Pfr, Fbq):
    coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))

pL, pL_vals = zip(*par_map.items())
la1 = par_map[l] / 4.                      # length of an axle
la2 = la1/1.8
coords_lam = sm.lambdify((*state_symbols, fb, *pL, la), coordinates,
        cse=True)

def init():
        xmin, xmax = -15, 15.
        ymin, ymax = -0.75, 12.5

        fig = plt.figure(figsize=(8, 8))
        ax  = fig.add_subplot(111)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect('equal')
        ax.grid()

        XX = np.linspace(xmin, xmax,  200)
        ax.fill_between(XX, ymin, street_lam(XX, par_map[a], par_map[b],
                par_map[c]-la2), color='grey', alpha=0.25)
        ax.fill_between(XX, street_lam(XX, par_map[a], par_map[b],
                par_map[c]+par_map[d]+la2), ymax, color='grey', alpha=0.25)

        ax.plot(XX, street_lam(XX, par_map[a], par_map[b], par_map[c]-la2),
                color='black')
        ax.plot(XX, street_lam(XX, par_map[a],
                par_map[b], par_map[c]+par_map[d]+la2), color='black')
        ax.vlines(-10.0, street_lam(-10.0, par_map[a], par_map[b],
                par_map[c]-la2), street_lam(-10, par_map[a], par_map[b],
                par_map[c]+par_map[d]+la2), color='red', linestyle='dotted')
        ax.vlines(10.0, street_lam(10, par_map[a], par_map[b], par_map[c]-la2),
                final_u, color='green', linestyle='-')
        ax.vlines(10.0, final_o,
                street_lam(10, par_map[a], par_map[b], par_map[c]+par_map[d]
                +la2), color='green', linestyle='-')
        ax.vlines(10.0, final_u, final_o, color='green', linestyle='dotted')


        line1, = ax.plot([], [], color='orange', lw=2)
        line2, = ax.plot([], [], color='red', lw=2)
        line3, = ax.plot([], [], color='magenta', lw=2)
        line4  = ax.quiver([], [], [], [], color='green', scale=10, width=0.004,
                headwidth=8)

        return fig, ax, line1, line2, line3, line4

# Function to update the plot for each animation frame
fig, ax, line1, line2, line3, line4 = init()
def update(t):
    message = (f'running time {t:.2f} sec \n The rear axle is red, the ' +
               f'front axle is magenta \n The driving/breaking force is green')
    ax.set_title(message, fontsize=12)

    coords = coords_lam(*state_sol(t), input_sol(t)[0], *pL_vals, la1)

    #   Pb, Dmc, Pf, Pbl, Pbr, Pfl, Pfr, Fbq
    line1.set_data([coords[0, 0], coords[0, 2]], [coords[1, 0], coords[1, 2]])
    line2.set_data([coords[0, 3], coords[0, 4]], [coords[1, 3], coords[1, 4]])
    line3.set_data([coords[0, 5], coords[0, 6]], [coords[1, 5], coords[1, 6]])

    line4.set_offsets([coords[0, 0], coords[1, 0]])
    line4.set_UVC(coords[0, 7] - coords[0, 0] , coords[1, 7] - coords[1, 0])
    return line1, line2, line3, line4, #line5, line6,

frames = np.linspace(t0, tf, int(fps * (tf - t0)))
animation = FuncAnimation(fig, update, frames=frames, interval=1000 / fps)


# Show the animation
display(HTML(animation.to_jshtml()))
plt.close()
